In [1]:
import pandas as pd

In [9]:
main_df = pd.DataFrame()
ratios = ['BTC-USD', 'LTC-USD', 'ETH-USD', 'BCH-USD']
for ratio in ratios:
    file = f"Data/crypto/{ratio}.csv"
    df = pd.read_csv(file, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    df.rename(columns={"close":f"{ratio}_close",
                       "volume":f"{ratio}_volume"}, inplace=True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
        
main_df

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.010010,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.000000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.750000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.000000,77.355759,870.000000,1.680500
...,...,...,...,...,...,...,...,...
1535214960,6713.140137,0.769891,58.020000,6.434783,279.359985,11.280577,531.479980,1.208560
1535215020,6714.520020,1.002652,58.009998,7.301921,279.359985,8.790519,531.479980,0.016868
1535215080,6714.520020,1.021925,58.020000,23.802017,279.369995,1.311763,531.469971,0.013854
